In [1]:
import pandas as pd

total_traces = 9

#txt_filename = "vgg19_arch.txt"
#csv_filename = "_vgg19_list1_data.csv"

#txt_filename = "mobilenet_arch.txt"
#csv_filename = "_mobilenet_list1_data.csv"

#txt_filename = "densenet_arch.txt"
#csv_filename = "_densenet_list1_data.csv"


txt_filename = "vgg16_arch.txt"
csv_filename = "_vgg16_list1_data.csv"

#txt_filename = "resnet_arch.txt"
#csv_filename = "_resnet_list1_data.csv"

#txt_filename = "alexnet_arch.txt"
#csv_filename = "_alexnet_list1_data.csv"


wordlist = list()
layerlist = list()

layers = {'(InputLayer)': 0, '(Lambda)': 0,
          '(MaxPooling2D)': 900, '(GlobalAveragePooling2D)': 910,
          '(Dense)': 500, '(Flatten)': 400, '(Dropout)': 700, '(Conv2D)': 600,
          '(BatchNormalization)': 200, '(Activation)': 300, '(ReLU)': 310,
          '(ZeroPadding2D)':920, '(Concatenate)': 800,
          '(DepthwiseConv2D)': 610, '(Add)': 100, '(Reshape)': 110
         }

In [2]:
with open('data/'+txt_filename, "r") as file:
    data = file.readlines()
    
    i = 0
    for line in data:
        word = line.split()
        if(i%2 == 0):
            if(len(word)>1):
                s = word[0] + " " + word[1]
            else:
                s = word[0]

            wordlist.append(s)
        i= i+1

In [3]:
wordlist.pop(0)
wordlist.pop(0)
wordlist.pop(0)
n = len(wordlist)

In [4]:
wordlist.pop(n-1)
wordlist.pop(n-2)
wordlist.pop(n-3)
print(wordlist)

['lambda (Lambda)', 'conv2d (Conv2D)', 'lambda_1 (Lambda)', 'conv2d_1 (Conv2D)', 'lambda_2 (Lambda)', 'max_pooling2d (MaxPooling2D)', 'lambda_3 (Lambda)', 'conv2d_2 (Conv2D)', 'lambda_4 (Lambda)', 'conv2d_3 (Conv2D)', 'lambda_5 (Lambda)', 'max_pooling2d_1 (MaxPooling2D)', 'lambda_6 (Lambda)', 'conv2d_4 (Conv2D)', 'lambda_7 (Lambda)', 'conv2d_5 (Conv2D)', 'lambda_8 (Lambda)', 'conv2d_6 (Conv2D)', 'lambda_9 (Lambda)', 'max_pooling2d_2 (MaxPooling2D)', 'lambda_10 (Lambda)', 'conv2d_7 (Conv2D)', 'lambda_11 (Lambda)', 'conv2d_8 (Conv2D)', 'lambda_12 (Lambda)', 'conv2d_9 (Conv2D)', 'lambda_13 (Lambda)', 'max_pooling2d_3 (MaxPooling2D)', 'lambda_14 (Lambda)', 'conv2d_10 (Conv2D)', 'lambda_15 (Lambda)', 'conv2d_11 (Conv2D)', 'lambda_16 (Lambda)', 'conv2d_12 (Conv2D)', 'lambda_17 (Lambda)', 'max_pooling2d_4 (MaxPooling2D)', 'lambda_18 (Lambda)', 'flatten (Flatten)', 'lambda_19 (Lambda)', 'dense (Dense)', 'lambda_20 (Lambda)', 'dropout (Dropout)', 'lambda_21 (Lambda)', 'dense_1 (Dense)', 'lambda

In [5]:

j = 1
for i in range(len(wordlist)):
    w = wordlist[i].split()
    if(w[1] != "(Lambda)"):
        layerlist.append(w[1])
    if w[1] in layers:
        pass
    else:
        layers[w[1]] = j
        j = j + 1

print(layers)

print(layerlist)


{'(InputLayer)': 0, '(Lambda)': 0, '(MaxPooling2D)': 900, '(GlobalAveragePooling2D)': 910, '(Dense)': 500, '(Flatten)': 400, '(Dropout)': 700, '(Conv2D)': 600, '(BatchNormalization)': 200, '(Activation)': 300, '(ReLU)': 310, '(ZeroPadding2D)': 920, '(Concatenate)': 800, '(DepthwiseConv2D)': 610, '(Add)': 100, '(Reshape)': 110}
['(Conv2D)', '(Conv2D)', '(MaxPooling2D)', '(Conv2D)', '(Conv2D)', '(MaxPooling2D)', '(Conv2D)', '(Conv2D)', '(Conv2D)', '(MaxPooling2D)', '(Conv2D)', '(Conv2D)', '(Conv2D)', '(MaxPooling2D)', '(Conv2D)', '(Conv2D)', '(Conv2D)', '(MaxPooling2D)', '(Flatten)', '(Dense)', '(Dropout)', '(Dense)', '(Dropout)', '(Dense)']


In [6]:
for i in range(total_traces):
    df = pd.read_csv("data/split/"+str(i+1)+csv_filename)
    df["label"] = ""
    r = len(df["branches"])
    #print(r)
    flag = -1
    c = 0
    d = 1

    flagmax = len(layerlist)

    for n in range(r):
        val = df["branches"][n] + df["cache_misses"][n] + df["cache_references"][n] + df["instructions"][n]
        if(val == 0):
            df["label"][n] = 0
            c = c + 1
            if c > 100 and d == 1:
                flag = flag + 1
                d = 0
                #print(flag, n)
                c = 0
        else:
            if flag < flagmax:
                l = layerlist[flag]
                df["label"][n] = layers[l]
                d = 1
                c = 0
            else:
                df["label"][n] = 0

    #df.describe()
    df.to_csv('data/labeled/auto_'+str(i+1)+csv_filename, encoding='utf-8', index=False)

<ipython-input-6-554d6b5d6a4f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][n] = 0
<ipython-input-6-554d6b5d6a4f>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][n] = layers[l]
<ipython-input-6-554d6b5d6a4f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][n] = 0
